<a href="https://colab.research.google.com/github/Shreyans06/LLMs-understanding/blob/main/tokenization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get the text file


In [100]:
import urllib
url =  ("https://raw.githubusercontent.com/rasbt/"
       "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
       "the-verdict.txt")
file_path = "verdict.txt"
urllib.request.urlretrieve(url , file_path)

('verdict.txt', <http.client.HTTPMessage at 0x7a72b90f8320>)

# Read the text file

In [101]:
with open("verdict.txt" , "r" , encoding="utf-8") as f:
  raw_text = f.read()

print("Total characters:" , len(raw_text))
print(raw_text[:50])

Total characters: 20479
I HAD always thought Jack Gisburn rather a cheap g


# Pre-process the text file

In [102]:
import re

pre_process = re.split(r'([,.:;?!"()_\']|--|\s)', raw_text)
pre_processed = [item.strip() for item in pre_process if item.strip()]
print(len(pre_processed))

4690


# Sample pre-processed data

In [103]:
print(pre_processed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


# Create a vocabulary

In [104]:
words = sorted(set(pre_processed))
vocab_size = len(words)
print(vocab_size)

1130


In [105]:
vocab = {token : integer for integer , token in enumerate(words)}
for i , item in enumerate(vocab.items()):
  print(item)
  if i >= 30:
    break


('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)


# Text Tokenizer

In [106]:
class TokenizerV1:
  def __init__(self , vocab):
    self.str_to_int = vocab
    self.int_to_str = {integer : string for string , integer in vocab.items() }

  def encode(self, text):
    preprocess = re.split(r'([,.()?_"!\']|--|\s)' , text)
    preprocessed = [ item.strip() for item in preprocess if item.strip()]
    ids = [ self.str_to_int[s] for s in preprocessed]
    return ids

  def decode(self , ids):
    text = " ".join([self.int_to_str[i] for i in ids])

    text = re.sub(r'\s+([,.()?!\'])' , r'\1' , text)
    return text

In [107]:
tokenizer = TokenizerV1(vocab)
text = """"It's the last he painted, you know,"
       Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [108]:
print(tokenizer.decode(ids))

" It' s the last he painted, you know, " Mrs. Gisburn said with pardonable pride.


In [109]:
tokens = sorted(list(set(pre_processed)))
tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token : integer for integer , token in enumerate(tokens)}

print(len(vocab.items()))

1132


In [110]:
for i, item in enumerate(list(vocab.items())[-5:]):
  print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [111]:
class TokenizerV2:
  def __init__(self, vocab):
    self.str_to_int = vocab
    self.int_to_str = { integer : string for string , integer in vocab.items()}

  def encode(self, text):
    pre_process = re.split(r'([,.()?_"!\']|--|\s)' , text)
    pre_processed = [ item.strip() for item in pre_process if item.strip()]

    pre_processed = [item if item in self.str_to_int
                     else "<|unk|>" for item in pre_processed]
    ids = [self.str_to_int[s] for s in pre_processed]
    return ids

  def decode(self , ids):
    text = " ".join([self.int_to_str[i] for i in ids])

    text = re.sub(r'\s+([.,:;?!"\'()])' , r'\1' , text)

    return text


In [112]:
sample_text1 = "Hi, How are you doing?"
sample_text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((sample_text1 , sample_text2))
print(text)

Hi, How are you doing? <|endoftext|> In the sunlit terraces of the palace.


In [113]:
tokenizer = TokenizerV2(vocab)
print(tokenizer.encode(text))

[1131, 5, 52, 169, 1126, 357, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]


In [114]:
print(tokenizer.decode(tokenizer.encode(text)))

<|unk|>, How are you doing? <|endoftext|> In the sunlit terraces of the <|unk|>.


In [115]:
from importlib.metadata import version
import tiktoken
print("tiktoken version:", version("tiktoken"))

tiktoken version: 0.12.0


In [116]:
tokenizer = tiktoken.get_encoding("gpt2")

In [117]:
text = (
    "Hello, do you like this place? <|endoftext|> In the sunlit "
    "terraces of some unknown place"
)
tokens = tokenizer.encode(text , allowed_special={"<|endoftext|>"})
print(tokens)

[15496, 11, 466, 345, 588, 428, 1295, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 617, 6439, 1295]


In [118]:
text = tokenizer.decode(tokens)
print(text)

Hello, do you like this place? <|endoftext|> In the sunlit terraces of some unknown place


In [119]:
sample_text = "UnitedSta tes"
tokens = tokenizer.encode(text = sample_text)
print(tokens)

[17013, 1273, 64, 256, 274]


In [120]:
sample_text = tokenizer.decode(tokens)
print(sample_text)

UnitedSta tes


In [121]:
with open("verdict.txt" , "r" , encoding="utf-8") as f:
  raw_text = f.read()

encoded_text = tokenizer.encode(raw_text)
print(len(encoded_text))

5145


In [122]:
encoded_sample = encoded_text[50:]

In [123]:
context_size = 4
x = encoded_sample[:context_size]
y = encoded_sample[1 : context_size + 1]
print(f"x : {x}")
print(f"y : {y}")

x : [290, 4920, 2241, 287]
y : [4920, 2241, 287, 257]


In [124]:
for i in range(1 , context_size + 1):
  context = encoded_sample[:i]
  desired = encoded_sample[i]
  print(context, "--->" , desired)

[290] ---> 4920
[290, 4920] ---> 2241
[290, 4920, 2241] ---> 287
[290, 4920, 2241, 287] ---> 257


In [125]:
for i in range(1 , context_size + 1):
  context = encoded_sample[:i]
  desired = encoded_sample[i]
  print(tokenizer.decode(context), "--->" , tokenizer.decode([desired] ))

 and --->  established
 and established --->  himself
 and established himself --->  in
 and established himself in --->  a


In [126]:
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
  def __init__(self , txt , tokenizer , max_length , stride):
    self.input_ids = []
    self.target_ids = []

    token_ids = tokenizer.encode(txt)

    for i in range(0 , len(token_ids) - max_length , stride):
      input_chunk = token_ids[i : i + max_length]
      target_chunk = token_ids[ i + 1 : i + 1 + max_length]
      self.input_ids.append(torch.tensor(input_chunk))
      self.target_ids.append(torch.tensor(target_chunk))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx] , self.target_ids[idx]

In [127]:
def create_dataloader_v1(txt, batch_size = 4 , max_length = 256 ,
                         stride = 128 , shuffle = True , drop_last = True , num_workers = 0):
  tokenizer = tiktoken.get_encoding("gpt2")
  dataset = GPTDatasetV1(txt , tokenizer , max_length , stride)
  dataloader = DataLoader(
      dataset,
      batch_size = batch_size,
      shuffle = shuffle,
      drop_last= drop_last,
      num_workers = num_workers
  )
  return dataloader

In [128]:
with open("verdict.txt" , "r" , encoding="utf-8") as f:
  raw_text = f.read()

dataloader = create_dataloader_v1(
    raw_text ,
    batch_size= 1 ,
    max_length= 4,
    stride = 1,
    shuffle = False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)


[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [129]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [130]:
dataloader = create_dataloader_v1(
    raw_text,
    batch_size= 8,
    max_length = 8 ,
    stride = 8,
    shuffle = False
)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n" , inputs)
print("Targets:\n" , targets)

Inputs:
 tensor([[   40,   367,  2885,  1464,  1807,  3619,   402,   271],
        [10899,  2138,   257,  7026, 15632,   438,  2016,   257],
        [  922,  5891,  1576,   438,   568,   340,   373,   645],
        [ 1049,  5975,   284,   502,   284,  3285,   326,    11],
        [  287,   262,  6001,   286,   465, 13476,    11,   339],
        [  550,  5710,   465, 12036,    11,  6405,   257,  5527],
        [27075,    11,   290,  4920,  2241,   287,   257,  4489],
        [   64,   319,   262, 34686, 41976,    13,   357, 10915]])
Targets:
 tensor([[  367,  2885,  1464,  1807,  3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632,   438,  2016,   257,   922],
        [ 5891,  1576,   438,   568,   340,   373,   645,  1049],
        [ 5975,   284,   502,   284,  3285,   326,    11,   287],
        [  262,  6001,   286,   465, 13476,    11,   339,   550],
        [ 5710,   465, 12036,    11,  6405,   257,  5527, 27075],
        [   11,   290,  4920,  2241,   287,   257,  4489

Convert to embedding vector

In [131]:
input_ids = torch.tensor([2 , 4 , 5 , 1])
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size , output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [132]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [133]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


In [137]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size , output_dim)

In [135]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text,
    batch_size = 8,
    max_length= max_length,
    stride = max_length,
    shuffle = False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Tokens ID:\n" , inputs)
print("Inputs shape:\n" , inputs.shape)

Tokens ID:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
Inputs shape:
 torch.Size([8, 4])


In [138]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [139]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length , output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [140]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
